# Notebook 1: Exploratory Data Analysis (EDA) and Data Preparation
## MIMIC-III Clinical Database for Recommendation Systems

**Author:** Data Science Team  
**Date:** November 2025  
**Objective:** Complete EDA with data loading, inspection, cleaning, and train-test split

---

## Workflow Overview

```
┌──────────────────────────────────────────────────────┐
│          EDA WORKFLOW PIPELINE                       │
├──────────────────────────────────────────────────────┤
│ 1. DATA LOADING                                      │
│    ↓                                                 │
│ 2. INITIAL INSPECTION (head, info, describe)        │
│    ↓                                                 │
│ 3. MISSING VALUES & DUPLICATES                       │
│    ↓                                                 │
│ 4. OUTLIERS DETECTION                                │
│    ↓                                                 │
│ 5. EXPLORATORY ANALYSIS (Distributions)             │
│    ↓                                                 │
│ 6. CORRELATION ANALYSIS                              │
│    ↓                                                 │
│ 7. CLASS BALANCING ASSESSMENT                        │
│    ↓                                                 │
│ 8. DATA CLEANING & PREPROCESSING                     │
│    ↓                                                 │
│ 9. TRAIN-TEST SPLIT & EXPORT                         │
│    ↓                                                 │
│ 10. FINAL SUMMARY REPORT                             │
└──────────────────────────────────────────────────────┘
```

## Section 1: Import Required Libraries

In [ ]:
# Import essential libraries for EDA and data preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import pickle
import json
from datetime import datetime
from scipy import stats
from scipy.stats import skew, kurtosis

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE

# Configure visualization
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print('✓ All libraries imported successfully!')
print(f'Execution timestamp: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

## Section 2: Load Dataset from MIMIC-III

In [ ]:
# Load MIMIC-III clinical database tables
import kagglehub
from kagglehub import KaggleDatasetAdapter

DATASET_NAME = 'ihssanened/mimic-iii-clinical-databaseopen-access'

def load_mimic_table(file_name, date_col=None):
    """Load a single MIMIC-III table using KaggleHub."""
    try:
        print(f'  Loading: {file_name}...', end=' ')
        df = kagglehub.load_dataset(
            KaggleDatasetAdapter.PANDAS,
            DATASET_NAME,
            file_name
        )
        if date_col and date_col in df.columns:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        print(f'✓ {df.shape[0]:,} rows × {df.shape[1]} columns')
        return df
    except Exception as e:
        print(f'✗ Error loading {file_name}')
        return None

print('\n=== Loading MIMIC-III Tables ===\n')
df_admissions = load_mimic_table('admissions.csv', date_col='admittime')
df_patients = load_mimic_table('patients.csv')
df_labevents = load_mimic_table('labevents.csv')
df_d_labitems = load_mimic_table('d_labitems.csv')

if df_admissions is None or df_patients is None:
    print('\n❌ Critical tables failed to load!')
else:
    print('\n✓ All critical tables loaded!')

## Section 3: Initial Data Inspection (head, info, describe)

In [ ]:
# Perform initial inspection of key tables
def inspect_table(df, name):
    print(f'\n{"="*80}')
    print(f'TABLE: {name.upper()}')
    print(f'{"="*80}')
    print(f'\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns')
    print(f'\nFirst 5 rows:')
    display(df.head())
    print(f'\nData types and missing values:')
    print(df.info())
    print(f'\nDescriptive statistics:')
    display(df.describe(include='all').round(2))

if df_admissions is not None:
    inspect_table(df_admissions, 'admissions')
if df_patients is not None:
    inspect_table(df_patients, 'patients')

## Section 4: Missing Values and Duplicates Detection

In [ ]:
# Analyze missing values
def analyze_missing(df, name):
    print(f'\n{"="*80}')
    print(f'Missing Values Analysis: {name.upper()}')
    print(f'{"="*80}')
    
    missing_summary = pd.DataFrame({
        'Column': df.columns,
        'Missing_Count': df.isnull().sum(),
        'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
    }).sort_values('Missing_Percentage', ascending=False)
    
    print(f'\nTotal missing: {df.isnull().sum().sum():,}')
    display(missing_summary.head(10))
    
    # Visualization
    missing_pct = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False).head(10)
    if len(missing_pct) > 0:
        fig, ax = plt.subplots(figsize=(12, 5))
        missing_pct.plot(kind='barh', ax=ax, color='coral')
        ax.set_xlabel('Missing Value Percentage (%)', fontweight='bold')
        ax.set_title(f'Missing Values Distribution: {name.upper()}', fontsize=12, fontweight='bold')
        plt.tight_layout()
        plt.show()
        print(f'\n**Interpretation:** This chart shows the percentage of missing values for each column.')
        print(f'High percentages indicate sparse data, which is common in clinical datasets.')

# Analyze duplicates
def analyze_duplicates(df, name):
    print(f'\n{"="*80}')
    print(f'Duplicates Analysis: {name.upper()}')
    print(f'{"="*80}')
    dup_count = df.duplicated().sum()
    print(f'Total duplicate rows: {dup_count}')
    print(f'Duplicate percentage: {(dup_count/len(df)*100):.2f}%')

if df_admissions is not None:
    analyze_missing(df_admissions, 'admissions')
    analyze_duplicates(df_admissions, 'admissions')

if df_labevents is not None:
    analyze_missing(df_labevents, 'labevents')

## Section 5: Outliers Detection using Box Plots

In [ ]:
# Detect and visualize outliers
if df_labevents is not None and 'valuenum' in df_labevents.columns:
    print('\nOutliers Detection in Laboratory Values')
    print('='*80)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Box plot for outliers
    sns.boxplot(y=df_labevents['valuenum'].dropna(), ax=axes[0], color='lightblue')
    axes[0].set_title('Box Plot: Laboratory Values (Outliers Detection)', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Laboratory Value')
    
    # Histogram with outlier regions
    Q1 = df_labevents['valuenum'].quantile(0.25)
    Q3 = df_labevents['valuenum'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    axes[1].hist(df_labevents['valuenum'].dropna(), bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[1].axvline(lower_bound, color='red', linestyle='--', linewidth=2, label='Outlier Boundary')
    axes[1].axvline(upper_bound, color='red', linestyle='--', linewidth=2)
    axes[1].set_title('Histogram: Laboratory Values with Outlier Boundaries', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Laboratory Value')
    axes[1].set_ylabel('Frequency')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    outlier_count = ((df_labevents['valuenum'] < lower_bound) | (df_labevents['valuenum'] > upper_bound)).sum()
    print(f'\n**Interpretation:**')
    print(f'• Total values: {len(df_labevents["valuenum"].dropna()):,}')
    print(f'• Outliers detected (IQR method): {outlier_count:,} ({outlier_count/len(df_labevents["valuenum"].dropna())*100:.2f}%)')
    print(f'• Lower bound: {lower_bound:.2f}')
    print(f'• Upper bound: {upper_bound:.2f}')
    print(f'• Outliers are values beyond 1.5×IQR from Q1 and Q3, indicated by the red dashed lines.')

## Section 6: Exploratory Analysis - Distributions and Histograms

In [ ]:
# Analyze distributions
print('\nExploratory Data Analysis: Variable Distributions')
print('='*80)

if df_labevents is not None and 'valuenum' in df_labevents.columns:
    print('\n[1] Distribution of Laboratory Values')
    lab_values = df_labevents['valuenum'].dropna()
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Histogram
    axes[0, 0].hist(lab_values, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0, 0].set_title('Histogram: Distribution of Laboratory Values', fontsize=11, fontweight='bold')
    axes[0, 0].set_xlabel('Laboratory Value')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # KDE plot
    lab_values.plot(kind='kde', ax=axes[0, 1], color='steelblue', linewidth=2)
    axes[0, 1].fill_between(axes[0, 1].get_lines()[0].get_xdata(), 
                            axes[0, 1].get_lines()[0].get_ydata(), alpha=0.3, color='steelblue')
    axes[0, 1].set_title('Kernel Density Estimate (KDE)', fontsize=11, fontweight='bold')
    axes[0, 1].set_xlabel('Laboratory Value')
    axes[0, 1].set_ylabel('Density')
    
    # Q-Q plot
    stats.probplot(lab_values, dist="norm", plot=axes[1, 0])
    axes[1, 0].set_title('Q-Q Plot: Normality Assessment', fontsize=11, fontweight='bold')
    
    # Summary statistics text box
    stats_text = f'Mean: {lab_values.mean():.2f}\nMedian: {lab_values.median():.2f}\nStd Dev: {lab_values.std():.2f}\n'
    stats_text += f'Min: {lab_values.min():.2f}\nMax: {lab_values.max():.2f}\nSkewness: {skew(lab_values):.2f}'
    axes[1, 1].text(0.1, 0.5, stats_text, fontsize=11, verticalalignment='center',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5), family='monospace')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f'\n**Interpretation:**')
    print(f'• The histogram shows the frequency distribution of laboratory values.')
    print(f'• The KDE plot provides a smooth estimate of the probability density.')
    print(f'• The Q-Q plot compares against normal distribution (points on diagonal = normal).')
    print(f'• Mean: {lab_values.mean():.2f}, Median: {lab_values.median():.2f}, Std Dev: {lab_values.std():.2f}')
    print(f'• Skewness: {skew(lab_values):.2f} (positive = right-skewed, negative = left-skewed)')

In [ ]:
# Patient demographics analysis
if df_patients is not None and 'gender' in df_patients.columns:
    print('\n[2] Distribution of Patient Demographics')
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gender distribution - Bar plot
    gender_counts = df_patients['gender'].value_counts()
    colors = ['#1f77b4', '#ff7f0e']
    bars = axes[0].bar(gender_counts.index, gender_counts.values, color=colors, edgecolor='black', alpha=0.8)
    axes[0].set_title('Bar Plot: Patient Gender Distribution', fontsize=11, fontweight='bold')
    axes[0].set_xlabel('Gender')
    axes[0].set_ylabel('Number of Patients')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height):,}', ha='center', va='bottom', fontweight='bold')
    
    # Gender distribution - Pie chart
    colors_pie = ['#1f77b4', '#ff7f0e']
    wedges, texts, autotexts = axes[1].pie(gender_counts.values, labels=gender_counts.index, 
                                            autopct='%1.1f%%', colors=colors_pie, startangle=90)
    axes[1].set_title('Pie Chart: Gender Proportion', fontsize=11, fontweight='bold')
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f'\n**Interpretation:**')
    print(f'• Total patients: {len(df_patients):,}')
    for gender, count in gender_counts.items():
        pct = count / len(df_patients) * 100
        print(f'• {gender}: {count:,} ({pct:.1f}%)')

## Section 7: Correlation Analysis Between Clinical Variables

In [ ]:
# Correlation analysis
print('\nCorrelation Analysis: Clinical Variables')
print('='*80)

if df_labevents is not None and 'subject_id' in df_labevents.columns:
    # Create pivot table
    df_lab_clean = df_labevents[['subject_id', 'itemid', 'valuenum']].dropna()
    pivot_data = df_lab_clean.pivot_table(
        index='subject_id',
        columns='itemid',
        values='valuenum',
        aggfunc='mean'
    )
    
    # Select top features
    top_items = pivot_data.notna().sum().nlargest(12).index
    pivot_subset = pivot_data[top_items].dropna(how='all').fillna(pivot_data[top_items].mean())
    
    if pivot_subset.shape[0] > 2:
        # Correlation matrix
        corr_matrix = pivot_subset.corr()
        
        fig, ax = plt.subplots(figsize=(13, 11))
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                   square=True, ax=ax, cbar_kws={'label': 'Pearson Correlation'}, 
                   vmin=-1, vmax=1, linewidths=0.5)
        ax.set_title('Correlation Matrix: Top 12 Laboratory Items', fontsize=12, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print(f'\n**Interpretation:**')
        print(f'• This heatmap shows pairwise Pearson correlations between the top laboratory items.')
        print(f'• Values close to +1 (dark red) indicate strong positive correlation.')
        print(f'• Values close to -1 (dark blue) indicate strong negative correlation.')
        print(f'• Values close to 0 (white) indicate weak or no linear correlation.')
        print(f'\n• Top 5 strongest correlations (excluding diagonal):')
        
        # Find top correlations
        corr_pairs = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr_pairs.append({
                    'Item1': corr_matrix.columns[i],
                    'Item2': corr_matrix.columns[j],
                    'Correlation': corr_matrix.iloc[i, j]
                })
        
        top_corrs = sorted(corr_pairs, key=lambda x: abs(x['Correlation']), reverse=True)[:5]
        for idx, corr in enumerate(top_corrs, 1):
            print(f'  {idx}. Items {corr["Item1"]} ↔ {corr["Item2"]}: r = {corr["Correlation"]:.3f}')

## Section 8: Class Balancing Assessment

In [ ]:
# Class balancing analysis
print('\nClass Balancing Assessment')
print('='*80)

if df_admissions is not None and 'hospital_expire_flag' in df_admissions.columns:
    # Get unique admissions per patient
    df_admit_unique = df_admissions.drop_duplicates(subset=['subject_id'], keep='first')
    mortality = df_admit_unique['hospital_expire_flag'].value_counts().sort_index()
    mortality_pct = mortality / len(df_admit_unique) * 100
    
    print(f'\nTarget Variable: Hospital Mortality (hospital_expire_flag)')
    print(f'Total patients: {len(df_admit_unique):,}')
    print(f'\nClass Distribution:')
    print(f'  Class 0 (Survived): {mortality[0]:,} ({mortality_pct[0]:.2f}%)')
    print(f'  Class 1 (Died): {mortality[1]:,} ({mortality_pct[1]:.2f}%)')
    
    imbalance_ratio = max(mortality.values) / min(mortality.values)
    print(f'\nImbalance Ratio: {imbalance_ratio:.2f}:1')
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar plot
    colors_bar = ['#2ecc71', '#e74c3c']  # Green for survived, red for died
    bars = axes[0].bar(['Survived (0)', 'Died (1)'], mortality.values, color=colors_bar, edgecolor='black', alpha=0.8)
    axes[0].set_title('Bar Plot: Hospital Mortality Distribution', fontsize=11, fontweight='bold')
    axes[0].set_ylabel('Number of Patients')
    axes[0].grid(axis='y', alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height):,}', ha='center', va='bottom', fontweight='bold')
    
    # Pie chart
    explode = (0.05, 0.1)
    axes[1].pie(mortality.values, labels=['Survived', 'Died'], autopct='%1.1f%%',
               colors=colors_bar, explode=explode, startangle=90, shadow=True)
    axes[1].set_title('Pie Chart: Mortality Proportion', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f'\n**Interpretation:**')
    if imbalance_ratio > 2:
        print(f'⚠️  SIGNIFICANT CLASS IMBALANCE DETECTED!')
        print(f'   Recommendation: Apply balancing techniques')
        print(f'   • SMOTE (Synthetic Minority Over-sampling): Recommended')
        print(f'   • Random Under-sampling: Alternative approach')
        print(f'   • Weighted loss functions: For model training')
    else:
        print(f'✓ Classes are relatively balanced. Balancing may not be necessary.')

## Section 9: Data Cleaning and Preprocessing

In [ ]:
# Data cleaning and preprocessing
print('\nData Cleaning and Preprocessing')
print('='*80)

# Step 1: Clean admissions
if df_admissions is not None:
    print('\n[Step 1] Cleaning Admissions Table')
    df_admit_clean = df_admissions.drop_duplicates()
    df_admit_first = df_admit_clean.drop_duplicates(subset=['subject_id'], keep='first')
    print(f'  Original records: {len(df_admissions):,}')
    print(f'  After deduplication: {len(df_admit_clean):,}')
    print(f'  First admission per patient: {len(df_admit_first):,}')

# Step 2: Create feature matrix from labevents
if df_labevents is not None:
    print('\n[Step 2] Creating Patient-Laboratory Feature Matrix')
    
    df_lab_valid = df_labevents[['subject_id', 'itemid', 'valuenum']].dropna(subset=['valuenum'])
    print(f'  Valid lab events: {len(df_lab_valid):,}')
    
    # Aggregate by mean
    df_lab_agg = df_lab_valid.groupby(['subject_id', 'itemid'])['valuenum'].mean().reset_index()
    
    # Pivot
    X_matrix = df_lab_agg.pivot(index='subject_id', columns='itemid', values='valuenum')
    print(f'  Feature matrix shape: {X_matrix.shape}')
    
    sparsity_pct = (X_matrix.isnull().sum().sum() / X_matrix.size * 100)
    print(f'  Sparsity: {sparsity_pct:.2f}%')
    
    # Impute with median
    X_matrix_imputed = X_matrix.fillna(X_matrix.median())
    print(f'  After imputation: {X_matrix_imputed.isnull().sum().sum()} missing values')
    
    # Select top features by variance
    top_features = X_matrix_imputed.var().nlargest(25).index
    X_final = X_matrix_imputed[top_features]
    print(f'  Selected top {len(top_features)} features by variance')

# Step 3: Prepare target variable
if df_admit_first is not None and 'hospital_expire_flag' in df_admit_first.columns:
    print('\n[Step 3] Preparing Target Variable')
    
    y_target = df_admit_first.set_index('subject_id')[['hospital_expire_flag']]
    
    # Align X and y
    common_patients = X_final.index.intersection(y_target.index)
    X_aligned = X_final.loc[common_patients]
    y_aligned = y_target.loc[common_patients, 'hospital_expire_flag'].astype(int)
    
    print(f'  Common patients: {len(common_patients):,}')
    print(f'  Final X shape: {X_aligned.shape}')
    print(f'  Final y shape: {y_aligned.shape}')
    print(f'  Target distribution:')
    print(f'    Class 0 (Survived): {(y_aligned == 0).sum():,}')
    print(f'    Class 1 (Died): {(y_aligned == 1).sum():,}')
else:
    X_aligned = X_final
    y_aligned = None
    print('\n⚠️  Target variable not available. Using features only.')

print('\n✓ Data preprocessing completed!')

## Section 10: Train-Test Split and Data Export

In [ ]:
# Train-test split and export
print('\nTrain-Test Split and Data Export')
print('='*80)

# Create output directory
output_dir = '/Users/michi/Desktop/IntercicloEstocasticos/ExamenPractico/processed_data'
os.makedirs(output_dir, exist_ok=True)
print(f'\n[1] Output directory: {output_dir}')

# Perform train-test split
print('\n[2] Stratified Train-Test Split (80-20)')

if y_aligned is not None:
    X_train, X_test, y_train, y_test = train_test_split(
        X_aligned, y_aligned,
        test_size=0.2,
        random_state=42,
        stratify=y_aligned
    )
else:
    X_train, X_test = train_test_split(
        X_aligned,
        test_size=0.2,
        random_state=42
    )
    y_train = None
    y_test = None

print(f'  Training set: {X_train.shape[0]:,} samples × {X_train.shape[1]} features')
print(f'  Test set: {X_test.shape[0]:,} samples × {X_test.shape[1]} features')

if y_train is not None:
    print(f'\n  Train class distribution:')
    print(f'    Survived: {(y_train == 0).sum():,} ({(y_train == 0).sum()/len(y_train)*100:.1f}%)')
    print(f'    Died: {(y_train == 1).sum():,} ({(y_train == 1).sum()/len(y_train)*100:.1f}%)')
    print(f'\n  Test class distribution:')
    print(f'    Survived: {(y_test == 0).sum():,} ({(y_test == 0).sum()/len(y_test)*100:.1f}%)')
    print(f'    Died: {(y_test == 1).sum():,} ({(y_test == 1).sum()/len(y_test)*100:.1f}%)')

# Standardize features
print('\n[3] Feature Standardization (StandardScaler)')
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
print(f'  Scaling completed (mean=0, std=1)')

# Export datasets
print('\n[4] Exporting Datasets')

# Training set
train_df = X_train_scaled_df.copy()
if y_train is not None:
    train_df['hospital_expire_flag'] = y_train

train_path = os.path.join(output_dir, 'data_train.csv')
train_df.to_csv(train_path)
print(f'  ✓ Training set: {train_path}')

# Test set
test_df = X_test_scaled_df.copy()
if y_test is not None:
    test_df['hospital_expire_flag'] = y_test

test_path = os.path.join(output_dir, 'data_test.csv')
test_df.to_csv(test_path)
print(f'  ✓ Test set: {test_path}')

# Complete dataset
complete_df = pd.concat([train_df, test_df])
complete_df['data_split'] = ['train'] * len(train_df) + ['test'] * len(test_df)

complete_path = os.path.join(output_dir, 'data_prepared.csv')
complete_df.to_csv(complete_path)
print(f'  ✓ Complete dataset: {complete_path}')

# Pickle format
pickle_path = os.path.join(output_dir, 'data_prepared.pkl')
with open(pickle_path, 'wb') as f:
    pickle.dump({
        'X_train': X_train_scaled_df,
        'X_test': X_test_scaled_df,
        'y_train': y_train,
        'y_test': y_test,
        'scaler': scaler,
        'feature_names': X_train.columns.tolist()
    }, f)
print(f'  ✓ Pickle file: {pickle_path}')

# Metadata
metadata = {
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'features': X_train.shape[1],
    'feature_names': X_train.columns.tolist(),
    'target_variable': 'hospital_expire_flag',
    'train_test_ratio': '80-20',
    'scaling_method': 'StandardScaler',
    'export_date': datetime.now().isoformat()
}

metadata_path = os.path.join(output_dir, 'data_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=4, default=str)
print(f'  ✓ Metadata: {metadata_path}')

print('\n✓ All datasets exported successfully!')

## Section 11: Final Summary Report

In [ ]:
# Final summary
print('\n' + '='*80)
print('EXPLORATORY DATA ANALYSIS - FINAL SUMMARY REPORT')
print('='*80)

print(f'\n📊 DATA SOURCES:')
print(f'  • MIMIC-III Clinical Database (Kaggle Hub)')
print(f'  • Tables: Admissions, Patients, Lab Events, Lab Items')

print(f'\n📈 DATASET OVERVIEW:')
print(f'  • Original admissions: {len(df_admissions) if df_admissions is not None else "N/A":,}')
print(f'  • Unique patients: {len(df_patients) if df_patients is not None else "N/A":,}')
print(f'  • Laboratory events: {len(df_labevents) if df_labevents is not None else "N/A":,}')

print(f'\n🔍 DATA QUALITY ASSESSMENT:')
print(f'  ✓ Missing values: Detected and documented')
print(f'  ✓ Duplicates: Removed {len(df_admissions) - len(df_admit_clean) if df_admissions is not None else 0}')
print(f'  ✓ Outliers: Detected using IQR method')
print(f'  ✓ Sparsity: Characterized and imputed')

print(f'\n📊 FINAL DATASET:')
print(f'  • Training samples: {len(X_train):,}')
print(f'  • Test samples: {len(X_test):,}')
print(f'  • Total features: {X_train.shape[1]}')
print(f'  • Train-Test split: 80-20')
print(f'  • Feature scaling: StandardScaler (mean=0, std=1)')

print(f'\n💾 EXPORTED FILES:')
print(f'  1. data_train.csv - Training dataset')
print(f'  2. data_test.csv - Test dataset')
print(f'  3. data_prepared.csv - Complete dataset with split indicator')
print(f'  4. data_prepared.pkl - Python pickle format')
print(f'  5. data_metadata.json - Metadata and data dictionary')
print(f'\n  📂 Location: {output_dir}')

print(f'\n✅ EDA PROCESS COMPLETED!')
print(f'⏰ Timestamp: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
print('='*80)